In [ ]:
!nvidia-smi

Mon Feb 28 06:46:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --quiet simpletransformers
!pip install --quiet tensorboardx
!pip install --quiet stop_words

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs

In [ ]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [ ]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


# Fetch Don't Patronize Me! data manager module

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [ ]:
from dont_patronize_me import DontPatronizeMe

In [ ]:
dpm = DontPatronizeMe('.', '.')

In [ ]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


# Task-2 Dataset

In [ ]:
pd.set_option('display.max_colwidth',1000)

In [ ]:
init_df = dpm.train_task2_df
init_df.tail()

,par_id,art_id,text,keyword,country,label
988,873,@@20374243,"Citing the fact that these kids who died at Gorakhpur 's Baba Raghav Das ( BRD ) Medical College and Farrukhabad district hospital belonged to the poor families , the Shiv Sena said instead of acting as a ' God ' to the poor ' , these government hospitals are proving to be a ' God of death ' for them .",poor-families,sg,"[1, 0, 0, 0, 1, 0, 0]"
989,10070,@@15573661,"Fern ? ndez was a well-known philanthropist who never forgot his motherland and was honoured by the former King of Spain , Juan Carlos , for his charitable deeds , including his work with disabled young people .",disabled,ng,"[1, 0, 0, 0, 0, 0, 0]"
990,6484,@@2559173,"Touched much by their plight , Commanding Officer and all ranks of the 7 Sri Lanka Light Infantry ( SLLI ) Regiment serving under the Security Fore Headquarters - Kilinochchi ( SFHQ-KLN ) , offered their financial and labour support for construction of a new house to a homeless family in Kanakapuram , Kilinochchi.",homeless,lk,"[1, 0, 0, 0, 0, 1, 0]"
991,6249,@@1947926,She reiterated her ministry 's commitment to put in place the necessary legal and policy framework to address all issues that affect women 's rights and gave a strong indication of hope for Ghanaian women .,women,gh,"[1, 0, 0, 0, 0, 0, 0]"
992,5149,@@1789214,"Preaching the sermon , the Dean of the St. Peter 's Cathedral , Very Rev. Seth Amoako-Adu observed that , the distasteful scenes at the Osu Children 's Home , which were aired recently , could have been avoided , if the Ghanaian society had shown a little more care to the orphans and vulnerable children at the home .",vulnerable,gh,"[0, 0, 0, 1, 0, 0, 0]"


In [ ]:
init_df.rename(columns = {'label':'labels'}, inplace = True)

In [ ]:
print(len(init_df))

993


## Adding more data

In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)

In [ ]:
rows2 = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  label = trids.label[idx]
  # select row from original dataset to retrieve the `text` value
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  rows2.append({
      'par_id':parid,
      'text':text,
      'label':label,
      'keyword':keyword
  })

In [ ]:
additional_df = pd.DataFrame(rows2)
additional_df.rename(columns = {'label':'labels'}, inplace = True)
# additional_df.head()

In [ ]:
# type(init_df['labels'][0])
import numpy as np
import ast
# lis = ast.literal_eval(additional_df['labels'][0])
# np.array(additional_df['labels'][0])

In [ ]:
additional_df.loc[:,"labels"] = additional_df.labels.apply(lambda x: np.array(ast.literal_eval(x)))

In [ ]:
# init_df.head()

In [ ]:
frames = [init_df[['text','keyword','labels']], additional_df[['text','keyword','labels']]]
df = pd.concat(frames, ignore_index=True)

In [ ]:
len(df)

9368

In [ ]:
import ast

np.fromstring(df['labels'][0][1:-1], dtype=int, sep=' ')
df['labels'] = df['labels'].apply(lambda x: np.fromstring(x[1:-1], dtype=int, sep=' '))

In [ ]:
class_dist = np.array([0,0,0,0,0,0,0])
# ct = 0
for i in range(len(df)):
  row = df.iloc[i]['labels']
  # if (row==class_dist).all():
  #   ct+=1
  class_dist+=row
# print(ct)

In [ ]:
class_dist

array([5114, 1413, 1532, 1682, 1361, 3307,  274])

# Paraphrase

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = model.to("cuda")

def paraphrase_instance(sentence):
  text =  "paraphrase: " + sentence + " </s>"
  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")

  outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      max_length=256,
      do_sample=True,
      no_repeat_ngram_size=2, 
      top_k=120,
      top_p=0.95,
      early_stopping=True,
      num_return_sequences=3
  )

  lines = []
  for output in outputs:
      line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if line.lower()!=sentence and line.lower() not in lines:
        lines.append(line.lower())
        # return line

  return lines

In [ ]:
paraphrase_df = df

In [ ]:
ctr = 0
for i in range(len(df)):
  # print(i)
  if 1 in df.iloc[i].labels:
    ctr+=1
    print(ctr)
    # paraphrased_sentence = paraphrase_instance(training_set1.iloc[i].text)
    paraphrased_list = paraphrase_instance(df.iloc[i].text)
    # # print("original text")
    # # print(training_set1.loc[i,'text'])
    # # print("paraphrased text")
    # # print(paraphrased_sentence)
    # paraphrased_row = {'text': paraphrased_sentence, 'keyword': training_set1.iloc[i].keyword, 'label': 1}
    for line in paraphrased_list:
      paraphrased_row = {'text': line, 'keyword': df.iloc[i].keyword, 'labels': df.iloc[i].labels}
      paraphrase_df = paraphrase_df.append(paraphrased_row, ignore_index = True)

In [ ]:
paraphrase_df.to_csv('task2paradf.csv', index=False)

In [ ]:
paraphrase_df

,text,keyword,labels
0,we also know that they can benefit by receiving counseling from someone who can help them understand that their feelings are normal and that their situation is not hopeless someone who can help them put their situation in perspective and help them communicate with others who could provide support someone knowledgeable about resources they can access someone who can help them plan for their needs and the needs of their child by developing either a parenting plan or an adoption plan,hopeless,"[1, 0, 0, 1, 0, 0, 0]"
1,pope francis washed and kissed the feet of muslim orthodox hindu and catholic refugees in a moving ceremony during holy thursday mass at a refugee centre on the outskirts of rome march 24,refugee,"[0, 1, 0, 0, 0, 0, 0]"
2,many refugees do n t want to be resettled anywhere let alone in the us,refugee,"[0, 0, 1, 0, 0, 0, 0]"
3,budding chefs like fred winston and angela in the kitchen are either homeless or recently re housed and desperately in need of a helping hand to enable them to re build their shattered lives,in-need,"[1, 0, 0, 1, 1, 1, 0]"
4,in a 90 degree view of his constituency one can see a high rise and a flyover while underneath it homeless people sleep on pavements as you can see my constituency is not confined to one socio economic group he said adding that apart from water another major concern for him is to improve the standard of public schools in the area,homeless,"[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...
14662,"she reiterated her ministry s commitment to put in place the necessary legal and policy framework to address all issues that affect women's rights, and gave a strong indication of hope for ghanaian women",women,"[1, 0, 0, 0, 0, 0, 0]"
14663,she reiterated her commitment to put in place the necessary legal and policy framework to address all issues that affect women's rights and gave a strong indication of hope for ghanaian women,women,"[1, 0, 0, 0, 0, 0, 0]"
14664,"the dean of st. peter's cathedral, very rev. seth amoako adu - said that the scandalous scenes at the osu elk children s home aired recently could have been avoided if the ghanaian society had shown n ° m ; d u",vulnerable,"[0, 0, 0, 1, 0, 0, 0]"
14665,the dean of st. peter s cathedral very rev seth amoako adu observed that the disgraceful scenes at the osu children's home which were aired recently could have been avoided if the afghan society had shown - albeit sadly — little more to orphans and vulnerable children at home.,vulnerable,"[0, 0, 0, 1, 0, 0, 0]"


In [ ]:
paraphrase_df = pd.read_csv('task2paradf.csv')

In [ ]:
old_df = df
df = paraphrase_df

In [ ]:
df

# Preprocess Text

In [ ]:
df.loc[:,"text"] = df.text.apply(lambda x : str.lower(x))

In [ ]:
import re
df.loc[:,"text"] = df.text.apply(lambda x : " ".join(re.findall('[\w]+',x)))

In [ ]:
from stop_words import get_stop_words
stop_words = get_stop_words('en')

def remove_stopWords(s):
    '''For removing stop words
    '''
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

df.loc[:,"text"] = df.text.apply(lambda x: remove_stopWords(x))

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

# Adding Keyword to Input text

In [ ]:
keyword_column = df["keyword"].copy()
 
# concatenating team with name column
# overwriting name column
df["text"]= df["text"].str.cat(keyword_column, sep =",")

In [ ]:
df[['text','labels']]

,text,labels
0,"plainly cultural latitude forgiveness extended less successful black hiphop artists back posed question now big day buckled will next marginal black performer antisocial lyrics financially vulnerable enough forced submission well next target turned odd future collective may politics little principle,vulnerable","[0, 0, 0, 0, 0, 0, 0]"
1,"everyone tacloban surrounding provinces hit typhoon storm surge still process living lives said ted tv patrol report still much need continuous help assistance,in-need","[0, 0, 0, 1, 1, 0, 0]"
2,"fr ron rolheiserwhenever best christians open churches homes poor endangered also long proud history wherein refugees homeless persons taken refuge believe jesus tells us last judgment twenty fiveth chapter gospel serve us well stand god end,homeless","[1, 0, 0, 0, 0, 0, 0]"
3,"christmas volunteers epuni baptist church will giving away klosen bread families need,in-need","[1, 1, 0, 0, 0, 0, 0]"
4,"karachi sindh chief minister syed qaim ali shah directed social welfare department establish centre elderly homeless people live respectfully without working hard even without stretching hands others,homeless","[1, 0, 0, 0, 1, 0, 0]"
...,...,...
14662,"journey tumultuous times offspring early chinese immigrants singapore weathered difficult years 1940 1960s events including japanese occupation took place,immigrant","[0, 0, 0, 0, 0, 0, 0]"
14663,"will continue support vulnerable people across society issuing public health alerts spells hot weather providing advice schools taking steps tackle overheating risks new homes,vulnerable","[0, 0, 0, 0, 0, 0, 0]"
14664,"many cases resultant effects type living elderly looking frail past usefulness unable work physically weak mentally slow disabled hopeless,hopeless","[0, 0, 1, 1, 0, 1, 0]"
14665,"women girls particularly vulnerable trade although also increasing reports children including young boys trafficked prostitutes,vulnerable","[0, 0, 0, 0, 0, 0, 0]"


# Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['labels'], test_size=0.2, random_state=42)

In [ ]:
frame = {'text':X_train, 'labels':y_train}
train_set = pd.DataFrame(frame)
frame = {'text':X_test, 'labels':y_test}
valid_set = pd.DataFrame(frame)

In [ ]:
# train_set['labels']

# Multilabel Classification

In [ ]:
# Optional model configuration
model_args = MultiLabelClassificationArgs(num_train_epochs=1, no_cache=True, overwrite_output_dir=True)

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=7,
    use_cuda=cuda_available,
    args=model_args
)

# Train the model
model.train_model(train_set)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(valid_set)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'c

  0%|          | 0/11733 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1467 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2934 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/367 [00:00<?, ?it/s]

In [ ]:
result

{'LRAP': 0.9253778362060573, 'eval_loss': 0.2649020173203718}

In [ ]:
model_outputs

array([[0.43530273, 0.05194092, 0.27563477, ..., 0.17431641, 0.49633789,
        0.03720093],
       [0.08990479, 0.01646423, 0.03582764, ..., 0.01542664, 0.07751465,
        0.00595856],
       [0.84082031, 0.60839844, 0.03521729, ..., 0.06164551, 0.09155273,
        0.03726196],
       ...,
       [0.04830933, 0.01646423, 0.02088928, ..., 0.01337433, 0.03125   ,
        0.00981903],
       [0.79785156, 0.1161499 , 0.24377441, ..., 0.27539062, 0.54882812,
        0.03607178],
       [0.06152344, 0.01495361, 0.02262878, ..., 0.01137543, 0.04202271,
        0.00634003]])

In [ ]:
len(wrong_predictions)

1329

In [ ]:
len(valid_set)

2934

In [ ]:
valid_set.head()

,text,labels
3207,"warmth envelopes disabled child upon discovery mute love excitement comes support loved ones revitalizes one s determination pursue osmanship start ghoulish,disabled","[0, 0, 0, 0, 1, 1, 0]"
4256,"real story hopelessly unqualified individual placed sensitive post,hopeless","[0, 0, 0, 0, 0, 0, 0]"
8830,"archdioceses cebu vancouver canada hosted table hope banquet saturday 23 january poor families also given bags containing canned sardines one kilo rice bottle water,poor-families","[1, 1, 0, 0, 0, 1, 0]"
6853,"mott due reduce australia s squad 15 13 monday said wait squad travelled sydney inspected north sydney pitch historic first women s day night test,women","[0, 0, 0, 0, 0, 0, 0]"
2495,"refugee boy refugee s person story life family accomplishments hardships hopes desire safety belonging,refugee","[1, 0, 0, 1, 0, 1, 0]"


In [ ]:
texlist = [line for line in valid_set['text']]
predictions, raw_outputs = model.predict(texlist)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2934 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

In [ ]:
predictions

# Predictions on the SemEval Test Dataset

## Train on the entire training set (train+valid set)

In [ ]:
# Using the same model configuration as before
model_args = MultiLabelClassificationArgs(num_train_epochs=1, no_cache=True, overwrite_output_dir=True)

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=7,
    args=model_args,
)

# Train the model
model.train_model(df)

In [ ]:
test_df = pd.read_csv('task4_test.tsv',sep='\t', names=["id", "art_id", "keyword", "country", "text"])
test_df.head()

,id,art_id,keyword,country,text
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to weaken Clinton and drive down her numbers in early voting states , where she is increasingly vulnerable . They are , in effect , doing Sanders 's dirty work for him while he avoids scrutiny ."
1,t_1,@@16397324,women,pk,"In most poor households with no education children are a matter of routine . The house maid industry is a sorry tale of dysfunctional families . The norm of these families is to make their women and children work while the men do not work , are on drugs , and either just abuse their wives or produce more children ."
2,t_2,@@16257812,migrant,ca,"The real question is not whether immigration is good for the country or bad for the country . The real question is the intent of the immigrant . It is a subtle point , and one that is very easy to miss ."
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population has grown by 4.1million from 2011 to 2015 ."
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken Copeland Ministries , had recently traveled abroad and contracted measles . Church-members who were not vaccinated were particularly vulnerable to catching the rare disease , according to Healthline ."


In [ ]:
test_df['keyword'].value_counts()

refugee          416
women            412
immigrant        400
vulnerable       400
in-need          399
homeless         394
hopeless         385
migrant          371
disabled         332
poor-families    323
Name: keyword, dtype: int64

In [ ]:
#Preprocess test data
test_df.loc[:,"text"] = test_df.text.apply(lambda x : str.lower(x))
test_df.loc[:,"text"] = test_df.text.apply(lambda x : " ".join(re.findall('[\w]+',x)))
test_df.loc[:,"text"] = test_df.text.apply(lambda x: remove_stopWords(x))

In [ ]:
#Adding keyword to text
keyword_column = test_df["keyword"].copy()
 
# concatenating team with name column
# overwriting name column
test_df["text"]= test_df["text"].str.cat(keyword_column, sep =",")

In [ ]:
test_df = test_df[['text']]

In [ ]:
test_df

,text
0,"meantime conservatives working weaken clinton drive numbers early voting states increasingly vulnerable effect sanders s dirty work avoids scrutiny,vulnerable"
1,"poor households education children matter routine house maid industry sorry tale dysfunctional families norm families make women children work men work drugs either just abuse wives produce children,women"
2,"real question whether immigration good country bad country real question intent immigrant subtle point one easy miss,migrant"
3,"total country s immigrant population grown 4 1million 2011 2015,migrant"
4,"members church part ken copeland ministries recently traveled abroad contracted measles church members vaccinated particularly vulnerable catching rare disease according healthline,vulnerable"
...,...
3827,"letter dated thursday european commission president jean claude juncker prime minister viktor orban said fences erected 2015 borders serbia croatia practically eliminated migrant flow hungary guard just country,migrant"
3828,"discovered poor families healthier kids collecting small crabs shrimp rice paddies grinding kids food feeding smaller frequent meals,poor-families"
3829,"married 19 milan emil badovinac immigrant malinci now bosnia herzegovina 12 years senior arranged marriage ten months later became mother emil sonny second son nicky born 1942,migrant"
3830,"united kingdom n t going devolve something seen movies like children men country literally shut borders rounded illegal immigrants become viciously anti immigrant s pure dystopia s brexit s people saying want country back s racist xenophobic s common sense uk still going allow immigration re still going trade nations europe re still going deal eu s end world folks given liberal media obsessed race n t shocked re smearing entire leave camp bunch xenophobic troglodytes,immigrant"


In [ ]:
test_list = test_df['text'].to_list()
test_list[:3]

['meantime conservatives working weaken clinton drive numbers early voting states increasingly vulnerable effect sanders s dirty work avoids scrutiny,vulnerable',
 'poor households education children matter routine house maid industry sorry tale dysfunctional families norm families make women children work men work drugs either just abuse wives produce children,women',
 'real question whether immigration good country bad country real question intent immigrant subtle point one easy miss,migrant']

In [ ]:
predictions, raw_outputs = model.predict(test_list)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3832 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

In [ ]:
predictions

# Storing results in text file

In [ ]:
valid_set['labels']

In [ ]:
labels2file(predictions, 'task2.txt')

In [ ]:
!cat gold2.txt | head -n 10

0,0,0,0,0,0,0
0,0,0,0,0,0,0
1,1,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
1,0,0,1,1,1,0
0,0,0,0,0,0,0
